<a href="https://colab.research.google.com/github/ElcioLS/-/blob/main/MeuProjeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [17]:
#Importando e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

GOOGLE_API_KEY="MINHA-API-KEY"
genai.configure(api_key=GOOGLE_API_KEY)


In [18]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


In [19]:
#Exemplo de embedding
title = "MEI - Micro Empreendedor Individual "
sample_text = ("Título: Como explorar todas as vantagens de ser um MEI?"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: criando protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.017758576, -0.053955946, 0.0013093012, 0.036171302, 0.059341162, -0.025739716, -0.009657333, -0.029286891, 0.010518639, 0.050384477, 0.026462901, -0.0050517744, -0.044508006, -0.039773446, 0.007715337, -0.041480456, 0.0035695778, -0.014912218, -0.029072288, -0.01817566, -0.00013521979, 0.010987845, -0.011001773, -0.08271141, -0.0028186128, -0.004591134, 0.04060882, -0.019904794, -0.052036524, 0.0671398, -0.030169569, 0.08642837, -0.056533374, 1.2496568e-05, -0.008844312, -0.070232145, -0.025860457, -0.06090794, -0.011180182, -0.011044321, 0.031824235, -0.06659675, -0.030993793, -0.04457461, -0.02085497, -0.015928244, 0.003402003, 0.0568879, 0.017268829, -0.06391901, 0.009537342, 0.024515076, 0.11398441, -0.050870556, 0.011233928, -0.0073407306, 0.060066972, -0.043252017, -0.018615544, 0.009927312, 0.011681839, 0.0054578385, 6.455116e-05, 0.04244284, -0.016074216, -0.04458489, -0.029380254, 0.019495737, 0.027488764, -0.00624809, -0.006110017, 0.0057576634, 0.06284281, -

In [ ]:

# Modelo de embedding
model = "models/embedding-001"

# Informações sobre MEI
DOCUMENT1 = {
    "Nome": "Limite de Faturamento MEI",
    "Descricao": "O limite de faturamento anual para um MEI em 2023 é de R$ 81.000. Se você ultrapassar esse limite, precisará se desenquadrar do MEI e optar por outro regime tributário."
}

DOCUMENT2 = {
    "Nome": "Declaração Anual MEI (DASN-SIMEI)",
    "Descricao": "Todo MEI precisa entregar a Declaração Anual do Simples Nacional (DASN-SIMEI) até o dia 31 de maio de cada ano. Essa declaração informa à Receita Federal o seu faturamento anual."
}

DOCUMENT3 = {
    "Nome": "Crédito para MEI",
    "Descricao": "Existem diversas linhas de crédito disponíveis para MEI, como microcrédito, empréstimos para capital de giro e financiamentos para compra de equipamentos. Consulte o Sebrae ou instituições financeiras para conhecer as opções."
}

DOCUMENT4 = {
    "Nome": "Treinamento e Capacitação para MEI",
    "Descricao": "O Sebrae oferece diversos cursos e workshops gratuitos para MEI em áreas como gestão financeira, marketing, vendas e planejamento. Aproveite essas oportunidades para desenvolver suas habilidades empresariais."
}

DOCUMENT5 = {
    "Nome": "Compra de Veículos para MEI",
    "Descricao": "Alguns programas de financiamento oferecem condições especiais para a compra de veículos para MEI, como taxas de juros mais baixas e prazos de pagamento mais longos."
}

DOCUMENT6 = {
    "Nome": "Reforma da Empresa para MEI",
    "Descricao": "É possível utilizar linhas de crédito específicas para reformar ou ampliar o seu espaço físico, como o programa de microcrédito do BNDES."
}

# Crie uma lista de documentos
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4, DOCUMENT5, DOCUMENT6]

# Crie um DataFrame a partir da lista de documentos
df = pd.DataFrame(documents)

# Função para gerar embeddings
def embed_fn(title, text):
    return genai.embed_content(model=model,
                              content=text,
                              title=title,
                              task_type="RETRIEVAL_DOCUMENT")["embedding"]

# Gere embeddings para cada documento
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Nome"], row["Descricao"]), axis=1)

# Função para gerar e buscar consulta
def gerar_e_buscar_consulta(consulta, base, model):
    embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]
    produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]["Descricao"]

# Configuração de geração de texto
generation_config = {
    "temperature": 0.7, # Ajuste para controlar a criatividade das respostas
    "candidate_count": 1
}

# Modelo para geração de texto
model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)

# Loop principal do chatbot
while True:
    consulta = input("Olá! Sou seu assistente contábil para MEI. O que você gostaria de saber? ")

    if consulta.lower() == "sair":
        break

    trecho = gerar_e_buscar_consulta(consulta, df, model)

    # Prompt para o modelo de linguagem
    prompt = f"Responda a seguinte pergunta de um MEI em um tom amigável e informativo: {consulta}. Use a seguinte informação na sua resposta: {trecho}"

    # Gere a resposta
    response = model_2.generate_content(prompt)
    print(response.text)

Olá! Sou seu assistente contábil para MEI. O que você gostaria de saber? como abrir uma empresa
Olá, empreendedor! Seja muito bem-vindo ao mundo dos MEIs!

Abrir uma empresa pode parecer um desafio, mas com as orientações certas, pode ser um processo tranquilo. Aqui estão alguns passos para ajudá-lo:

1. **Escolha seu nome empresarial:** Ele deve ser único e fácil de lembrar.
2. **Defina seu ramo de atividade:** Escolha o CNAE (Código Nacional de Atividades Econômicas) que melhor descreve o que você faz.
3. **Formalize sua empresa:** Faça o registro no Portal do Empreendedor.
4. **Obtenha o CNPJ:** É o número de identificação da sua empresa.
5. **Inscreva-se no MEI:** Faça a inscrição no Portal do Simples Nacional.

Além disso, não deixe de aproveitar os cursos e workshops gratuitos oferecidos pelo Sebrae para MEIs. Eles são uma ótima oportunidade para você desenvolver suas habilidades empresariais e ter sucesso no seu negócio.

Se precisar de mais informações, não hesite em entrar em 